In [1]:
import os

In [2]:
os.listdir("/")

['initrd.img',
 'srv',
 'usr',
 'root',
 'media',
 'lib',
 'lost+found',
 'boot',
 'sys',
 'var',
 'opt',
 'proc',
 'home',
 'bin',
 'vmlinuz.old',
 'dev',
 'mnt',
 'run',
 'sbin',
 'lib64',
 'initrd.img.old',
 'data',
 'snap',
 'vmlinuz',
 'tmp',
 'etc']